In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
from statsmodels.stats.outliers_influence import variance_inflation_factor # VIF
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/wage2.csv')
df['age_urban'] = df['age']*df['urban']
df['IQ_urban']=df['IQ']*df['urban']
df

,wage,hours,IQ,KWW,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage,age_urban,IQ_urban
0,769,40,93,35,12,11,2,31,1,0,0,1,1,2.0,8.0,8.0,6.645091,31,93
1,808,50,119,41,18,11,16,37,1,0,0,1,1,NaN,14.0,14.0,6.694562,37,119
2,825,40,108,46,14,11,9,33,1,0,0,1,1,2.0,14.0,14.0,6.715384,33,108
3,650,40,96,32,12,13,7,32,1,0,0,1,4,3.0,12.0,12.0,6.476973,32,96
4,562,40,74,27,11,14,5,34,1,0,0,1,10,6.0,6.0,11.0,6.331502,34,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,520,40,79,28,16,6,1,30,1,1,1,0,0,1.0,11.0,NaN,6.253829,0,0
931,1202,40,102,32,13,10,3,31,1,0,1,1,7,7.0,8.0,6.0,7.091742,31,102
932,538,45,77,22,12,12,10,28,1,1,1,0,9,NaN,7.0,NaN,6.287858,0,0
933,873,44,109,25,12,12,12,28,1,0,1,0,1,1.0,NaN,11.0,6.771935,0,0


In [4]:
wage_eq12 = smf.ols(formula = 'np.log(wage)~ age + urban+age_urban + IQ+IQ_urban', data = df).fit()

In [5]:
info_dict = {'No. Observations': lambda x: '{0:d}'.format(int(x.nobs)), 
             'Residual Std. Error': lambda x: "%#8.3f" % x.mse_resid**0.5,
             'F-statistic': lambda x: '{:.3f}'.format(x.fvalue), 
             'Prob (F-statistic)': lambda x: '{:.3f}'.format(x.f_pvalue)
             }
print(summary_col(wage_eq12, float_format='%.3f', stars=True, info_dict = info_dict))


                    np.log(wage)
--------------------------------
Intercept           4.967***    
                    (0.291)     
age                 0.025***    
                    (0.007)     
urban               0.176       
                    (0.351)     
age_urban           -0.001      
                    (0.009)     
IQ                  0.009***    
                    (0.002)     
IQ_urban            0.000       
                    (0.002)     
R-squared           0.167       
R-squared Adj.      0.163       
No. Observations    935         
Residual Std. Error 0.385       
F-statistic         37.282      
Prob (F-statistic)  0.000       
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [6]:
df = summary_params(wage_eq12, alpha=0.05)
df['significance'] = df.apply(lambda x: 'Значим' if x['P>|t|']<0.05 else 'Незначим', axis=1)
df

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975],significance
Intercept,4.966571,0.291084,17.062320,5.449554e-57,4.395313,5.537830,Значим
age,0.024731,0.007394,3.344740,8.565624e-04,0.010220,0.039242,Значим
urban,0.175892,0.350904,0.501255,6.163103e-01,-0.512763,0.864548,Незначим
age_urban,-0.001056,0.008850,-0.119381,9.049993e-01,-0.018424,0.016311,Незначим
IQ,0.008538,0.001536,5.558691,3.551727e-08,0.005523,0.011552,Значим
IQ_urban,0.000390,0.001834,0.212682,8.316220e-01,-0.003209,0.003989,Незначим
